In [1]:
import pandas as pd
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error



In [2]:
#Gets the data from the scraper and deletes the index column
stats = pd.read_csv('total.csv')
del stats['Unnamed: 0']

In [3]:
#Gets Predidctor variables
'''
predictors = ['G', 'MP', 'FG%', '2P%', '3P%', 'FT%', 'PTS', 'TRB',
       'AST', 'STL', 'BLK', 'PER', 'WS', 'BPM', 'Year', 'Height w/o Shoes',
       'Height w/ Shoes', 'Weight', 'Wingspan', 'Standing Reach', 'Body Fat %',
       'Hand Length', 'Hand Width', 'Age']

columns = list(stats.columns)
predictors = columns.pop(['Player','Pos', 'Year', 'Pick'])
'''

columns_to_drop = ['SEASON',
 'PLAYER_ID',
 'FIRST_NAME',
 'LAST_NAME',
 'Player',
 'POSITION',
 'PER',
 'BPM',
 'Year',
 'Pick',
 'Age',
 'Pos',
 'STANDING_REACH_FT_IN']

predictors = (stats.drop(columns = columns_to_drop)).columns

#Training data from 2013-2018, testing data from 2019, model with Ridge

train = stats[stats["Year"] < 2020]
test = stats[stats['Year'] == 2020]
reg = Ridge(alpha = 0.01)
reg.fit(train[predictors], train['Pick'])

#Gets predictions for 2019 in the form of a df as well as mean squared error as an error metric for the predictions
predictions = reg.predict(test[predictors])
predictions = pd.DataFrame(predictions, columns = ['Predictions'], index = test.index)
combination = pd.concat([test[['Player', 'Pick']], predictions], axis = 1)
mse = mean_squared_error(combination['Pick'], combination['Predictions'])


In [4]:
#Displays rank which refers to each players draft position relative to one another in the given dataset as well as the predicted rank and difference
combination = combination.sort_values('Pick', ascending = True)
combination['RK'] = list(range(1, combination.shape[0]+1))

combination = combination.sort_values('Predictions', ascending = True)
combination['Predicted_RK'] = list(range(1, combination.shape[0]+1))
combination['Difference'] = combination['Predicted_RK'] - combination['RK']

In [5]:
def choose_year(year):
    train = stats[stats["Year"] < year]
    test = stats[stats['Year'] == year]
    reg = Ridge(alpha = 0.01)
    reg.fit(train[predictors], train['Pick'])
    predictions = reg.predict(test[predictors])
    predictions = pd.DataFrame(predictions, columns = ['Predictions'], index = test.index)
    combination = pd.concat([test[['Player', 'Pick']], predictions], axis = 1)
    mse = mean_squared_error(combination['Pick'], combination['Predictions'])
    return mse

    

In [6]:
year = input("Enter the draft class you would like to predict: ")

Enter the draft class you would like to predict: 2020


In [7]:
choose_year(int(year))

267.9941395467494